In [ ]:
import json
import pandas as pd
import numpy as np
import string
import re

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',-6)

In [3]:
tweets=pd.read_excel("combined_csv_v2.xlsx")

In [4]:


# read the file stored
tweets_data_path = 'tweets.json'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue


In [5]:
male=pd.read_excel('C:/Users/ibrahim/Desktop/Gp1/gender/male/male.xlsx')
female=pd.read_excel('C:/Users/ibrahim/Desktop/Gp1/gender/female/female.xlsx')

In [6]:
tweets.head()

,GUID,Date (CST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,Emotion,Source,Gender,Posts,Followers,Following,Influence Score,Post Title,Post Type,Image URL,Brand
0,1081815582889176960,2019-01-06 01:31:48,http://twitter.com/aboaljod59/status/1081815582889177088,@aboaljod59 @LuayPrince هذه المعلومات الي شفتها حق صب واي بالجامعة https://t.co/Gj9OwBYV1q,@aboaljod59,aboaljod59,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,547,35,376,0.010342,@aboaljod59 @LuayPrince هذه المعلومات الي شفتها حق صب واي بالجامعة https://t.co/Gj9OwBYV1q,Reply,NaN,NaN
1,1085259913788576000,2019-01-15 13:38:21,http://twitter.com/noudikh/status/1085259913788575744,@tariq_man_ @sectionb_sa اففف ياحلوو البرجر حقهممم 💔💔💔,@noudikh,العنود المساعد💙🐣,Saudi Arabia,Ar Riyad,Riyadh,Neutral,Unclassified,Twitter,NaN,32286,789,361,NaN,@tariq_man_ @sectionb_sa اففف ياحلوو البرجر حقهممم 💔💔💔,Reply,NaN,NaN
2,1092097568463565056,2019-02-03 10:28:45,http://twitter.com/__muha97/status/1092097568463564800,صب واي #ماك_او_كودو,@__muha97,نسّاي,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,1434,115,470,NaN,صب واي \n #ماك_او_كودو,Tweet,NaN,NaN
3,1090908449523814016,2019-01-31 03:43:37,http://twitter.com/abuhussien20201/status/1090908449523814400,RT @SubwaySaudi ودك بشيء خفيف ولذيذ 😋 #صب_واى_السعودية بانتظارك 😎👌 https://t.co/gI7DjXAyLy,@abuhussien20201,abuhussien2020,NaN,NaN,NaN,Positive,Unclassified,Twitter,NaN,67310,634,2027,NaN,RT @SubwaySaudi ودك بشيء خفيف ولذيذ 😋 #صب_واى_السعودية بانتظارك \n😎👌 https://t.co/gI7DjXAyLy,Retweet,NaN,NaN
4,1082695506277613952,2019-01-08 11:48:18,http://twitter.com/monakhaled88/status/1082695506277613571,@MaestroPizzaKSA #مسترها تم https://t.co/MO7CpgudYC,@monakhaled88,منى بنت خالد,Saudi Arabia,Ar Riyad,Riyadh,Neutral,Unclassified,Twitter,NaN,9736,83,510,NaN,@MaestroPizzaKSA #مسترها تم https://t.co/MO7CpgudYC,Reply,NaN,NaN


In [7]:
len(female)

1638

In [8]:

def remove_emoji(string):

    emoji_pattern = re.compile("["

                           u"\U0001F600-\U0001F64F"  # emoticons

                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs

                           u"\U0001F680-\U0001F6FF"  # transport & map symbols

                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)

                           u"\U00002702-\U000027B0"

                           u"\U000024C2-\U0001F251"

                           "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', string)

In [9]:

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)# Remove the punctuations
    return text.translate(translator)

In [10]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("[ َ ً ُ ٌ ِ ٍ ْ ّ ]"," ", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text.lower()

In [11]:
def cleanName(Name):
    
    if(Name.find(" ")!=-1):
        Name=Name[:Name.find(" ")] # To remove the last name
    Name=remove_punctuations(Name) # Remove the punctuations and emoji from name
    Name=remove_emoji(Name)
    Name=normalize_arabic(Name)
    return Name.lower()

In [12]:
#start processing the tweet
def processPost(tweet):

    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)
    
    # normalize the tweet
    tweet= normalize_arabic(tweet)
    
    # remove repeated letters
    tweet=remove_repeating_char(tweet)
    
    return tweet

In [13]:
# remove repeated letters
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

'saja🦋'

In [14]:
tweets["clean text"] = tweets['Contents'].apply(lambda x: processPost(str(x)))

In [15]:
tweets['clean Name']=tweets['Name'].apply(lambda x :cleanName(str(x)))

In [16]:
tweets['clean Name'].head(50)

0     aboaljod59    
1     العنود        
2     نس اي         
3     abuhussien2020
4     مني           
5     تميمي         
6     نويتش         
7     امل           
8     mohammad      
9     ‏اسال         
10    abdulaziz     
11    س             
12    ashwag        
13    rahaf         
14    منيره         
15    meddi         
16    v             
17    ridba9        
18    kroom         
19    مني           
20    salwa         
21    سنيكرز        
22    ahmed         
23    i             
24    موفق          
25    omhanoof      
26    r             
27    م ن يره       
28    اذكروا        
29    محمد          
30    tarip         
31    slmmunh       
32    عبق           
33    ابراهيم       
34    a7med         
35    s             
36    mutasem       
37    moh           
38    عيونالقلب     
39    steaky        
40    ه ال ه        
41    ابو           
42    احمد          
43    عبدالله       
44                  
45    ohood         
46    almanara      
47    دليل   

In [17]:
male=male.name.tolist()
female=female.name.tolist()

In [18]:
tweets.count()

GUID               146470
Date (CST)         146470
URL                65530 
Contents           146453
Author             146470
Name               146451
Country            64771 
State/Region       42560 
City/Urban Area    39196 
Category           146470
Emotion            146470
Source             146470
Gender             26517 
Posts              146470
Followers          146470
Following          146470
Influence Score    14664 
Post Title         146453
Post Type          146470
Image URL          0     
Brand              736   
clean text         146470
clean Name         146470
dtype: int64

In [19]:
tweets['gender1']=tweets['clean Name'].apply(lambda x : 'M' if x in male else "F" if x in female else "Known") #Determain the gender

In [20]:
tweets[tweets['gender1']=='Known'].count()

GUID               84848
Date (CST)         84848
URL                38339
Contents           84837
Author             84848
Name               84829
Country            34849
State/Region       21926
City/Urban Area    20104
Category           84848
Emotion            84848
Source             84848
Gender             6605 
Posts              84848
Followers          84848
Following          84848
Influence Score    9161 
Post Title         84837
Post Type          84848
Image URL          0    
Brand              435  
clean text         84848
clean Name         84848
gender1            84848
dtype: int64

In [21]:
Known=tweets[tweets['gender1']=='Known']

In [22]:
Known=Known['clean Name'].value_counts()

In [23]:
# create a list to capture all of the tweets
tweets_list = []

for tweet in tweets_data:
    
        tweet_id = tweet['id']
        
        whole_tweet = tweet['text']
         
        name = tweet['user']['name']
        
        lang= tweet['lang']
        
        if tweet['place'] != None:
            country = tweet['place']['country'] 
        else:
            country= 'None'
        
        only_url = whole_tweet[whole_tweet.find('https'):]
        url = only_url
        
        favorite_count = tweet['favorite_count']
         
        retweet_count = tweet['retweet_count']
        
        created_at = tweet['created_at']
        
        whole_source = tweet['source']
        
        only_device = whole_source[whole_source.find('rel="nofollow">') + 15:-4]
        source = only_device
        
        retweeted_status = tweet['retweeted_status'] = tweet.get('retweeted_status', 'Original tweet')
        if retweeted_status != 'Original tweet':
            retweeted_status = 'This is a retweet'

        tweets_list.append({'tweet_id': str(tweet_id),
                            'name':str(name),
                            'text' : str(whole_tweet),
                            'lang' : str(lang),
                            'country' : str(country),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'url': url,
                             'created_at': created_at,
                             'source': source,
                             'retweeted_status': retweeted_status,
                            })
       

In [24]:
# #create data frame for tweets
# tweets = pd.DataFrame(tweets_list, columns = ['tweet_id','name','text','lang','country', 
#                                               'favorite_count', 'retweet_count', 'created_at',
#                                               'source', 'retweeted_status', 'url'])


In [25]:
len(tweets)

146470

In [26]:
tweets.loc[7400:7500]

,GUID,Date (CST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,Emotion,Source,Gender,Posts,Followers,Following,Influence Score,Post Title,Post Type,Image URL,Brand,clean text,clean Name,gender1
7400,1097550482552381056,2019-02-18 11:36:41,http://twitter.com/fi11ii/status/1097550482552381440,@Taghroooda فيه الجلسات حول الرياض بارك حلوه .. ستيك هاوس .. مادو الي فالتحليه فتح .. سنتريا .. اضواء المساء,@fi11ii,باتريك #الشاهي☕️,Saudi Arabia,Ar Riyad,Riyadh,Positive,Unclassified,Twitter,NaN,2582,73,401,NaN,@Taghroooda فيه الجلسات حول الرياض بارك حلوه .. ستيك هاوس .. مادو الي فالتحليه فتح .. سنتريا .. اضواء المساء,Reply,NaN,NaN,taghroda فيه الجلسات حول الرياض بارك حلوه ستيك هاوس مادو الي فالتحليه فتح سنتريا اضواء المساء,باتريك,Known
7401,1094545436239646976,2019-02-10 04:35:42,http://twitter.com/A9eel82/status/1094545436239646720,@HamburginiSA ياخي افتحوا فرع في جدة,@A9eel82,اصيل,Saudi Arabia,Makkah,Jeddah,Neutral,Unclassified,Twitter,NaN,2114,93,510,NaN,@HamburginiSA ياخي افتحوا فرع في جدة,Reply,NaN,NaN,hamburginisa ياخي افتحوا فرع في جده,اصيل,M
7402,1099028431931691008,2019-02-22 13:29:31,http://twitter.com/FaisalDawood16/status/1099028431931691008,عزيزي، البيتزا شي 🇮🇹 والمفطح شي 🇸🇦 مو لازم نخلط الامور!! #لغوصه #مايسترو #بيتزا #ثقف_نفسك,@FaisalDawood16,Faisal Dawood,NaN,NaN,NaN,Negative,Unclassified,Twitter,M,92,20,145,NaN,عزيزي،\n البيتزا شي 🇮🇹\nوالمفطح شي 🇸🇦\nمو لازم نخلط الامور!!\n #لغوصه #مايسترو #بيتزا #ثقف_نفسك,Tweet,NaN,NaN,عزيزي البيتزا شي 🇮🇹 والمفطح شي 🇸🇦 مو لازم نخلط الامور لغوصه مايسترو بيتزا ثقفنفسك,faisal,M
7403,1093822031492275968,2019-02-08 04:41:09,http://twitter.com/B_vlinder/status/1093822031492276224,@zeze00042 @CareemKSA طريقة الطلب اختاروا فئة بيتزا نقطة الانطلاق مايسترو بيتزا طبعا تختاروا أقرب فرع لكم أو بكيفكم الفرع اللي تبون بعدين نقطة الوصول اللي هي مكان تواجدك بيطلع لك القيمة صفر والله يرزقني ويرزقكم من واسع فضله,@B_vlinder,B.vlinder,Saudi Arabia,Ar Riyad,Riyadh,Neutral,Unclassified,Twitter,NaN,3312,270,578,NaN,@zeze00042 @CareemKSA طريقة الطلب \nاختاروا فئة بيتزا \nنقطة الانطلاق مايسترو بيتزا طبعا تختاروا أقرب فرع لكم أو بكيفكم الفرع اللي تبون\nبعدين نقطة الوصول اللي هي مكان تواجدك \nبيطلع لك القيمة صفر \nوالله يرزقني ويرزقكم من واسع فضله,Reply,NaN,NaN,zeze042 caremksa طريقه الطلب اختاروا فئه بيتزا نقطه الانطلاق مايسترو بيتزا طبعا تختاروا اقرب فرع لكم او بكيفكم الفرع الي تبون بعدين نقطه الوصول الي هي مكان تواجدك بيطلع لك القيمه صفر واله يرزقني ويرزقكم من واسع فضله,bvlinder,Known
7404,1103387821237304960,2019-03-06 14:12:11,http://twitter.com/noooormd84404/status/1103387821237305344,@Hanan3omr @LaughWithMedia @MaestroPizzaKSA هذه سخنوها مكرويف,@noooormd84404,غربة ♋️ إحساس,Saudi Arabia,NaN,NaN,Neutral,Unclassified,Twitter,NaN,10346,456,360,NaN,@Hanan3omr @LaughWithMedia @MaestroPizzaKSA هذه سخنوها مكرويف,Reply,NaN,NaN,hanan3omr laughwithmedia maestropizaksa هذه سخنوها مكرويف,غربه,Known
7405,1104739997058507008,2019-03-10 08:45:15,http://twitter.com/hantamy55/status/1104739997058506754,@crazy_zoon @LaughWithMedia @MaestroPizzaKSA @Saad_Maestro حاجة تكسف..!! https://t.co/7kbI5K1XlW,@hantamy55,ابو عبدالرحمن,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,21919,235,490,NaN,@crazy_zoon @LaughWithMedia @MaestroPizzaKSA @Saad_Maestro حاجة تكسف..!! https://t.co/7kbI5K1XlW,Reply,NaN,NaN,crazyzon laughwithmedia maestropizaksa sadmaestro حاجه تكسف,ابو,Known
7406,1092724237087969024,2019-02-05 03:58:54,http://twitter.com/hasmoh567/status/1092724237087969280,@PainkillerQ8 @SUBWAYArabia @SubwaySaudi @SUBWAY دجاج مشوي مع جبن ابيض مع بصل مع خيار زيتون اسود بس شوي مع طماطم اهم شي وخس وصلصلة بصل حلو وقطرة خل,@hasmoh567,Hasan Mohammed,NaN,NaN,NaN,Neutral,Unclassified,Twitter,M,165,7,22,NaN,@PainkillerQ8 @SUBWAYArabia @SubwaySaudi @SUBWAY دجاج مشوي مع جبن ابيض مع بصل مع خيار زيتون اسود بس شوي مع طماطم اهم شي وخس وصلصلة بصل حلو وقطرة خل,Reply,NaN,NaN,painkilerq8 subwayarabia subwaysaudi subway دجاج مشوي مع جبن ابيض مع بصل مع خيار زيتون اسود بس شوي مع طماطم اهم شي وخس وصلصله بصل حلو وقطره خل,hasan,M
7407,110

In [27]:
tweets.drop_duplicates(inplace=True)
tweets.reset_index(drop=True,inplace=True)

In [28]:
tweets.loc[7400:7500]

,GUID,Date (CST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,Emotion,Source,Gender,Posts,Followers,Following,Influence Score,Post Title,Post Type,Image URL,Brand,clean text,clean Name,gender1
7400,1097550482552381056,2019-02-18 11:36:41,http://twitter.com/fi11ii/status/1097550482552381440,@Taghroooda فيه الجلسات حول الرياض بارك حلوه .. ستيك هاوس .. مادو الي فالتحليه فتح .. سنتريا .. اضواء المساء,@fi11ii,باتريك #الشاهي☕️,Saudi Arabia,Ar Riyad,Riyadh,Positive,Unclassified,Twitter,NaN,2582,73,401,NaN,@Taghroooda فيه الجلسات حول الرياض بارك حلوه .. ستيك هاوس .. مادو الي فالتحليه فتح .. سنتريا .. اضواء المساء,Reply,NaN,NaN,taghroda فيه الجلسات حول الرياض بارك حلوه ستيك هاوس مادو الي فالتحليه فتح سنتريا اضواء المساء,باتريك,Known
7401,1094545436239646976,2019-02-10 04:35:42,http://twitter.com/A9eel82/status/1094545436239646720,@HamburginiSA ياخي افتحوا فرع في جدة,@A9eel82,اصيل,Saudi Arabia,Makkah,Jeddah,Neutral,Unclassified,Twitter,NaN,2114,93,510,NaN,@HamburginiSA ياخي افتحوا فرع في جدة,Reply,NaN,NaN,hamburginisa ياخي افتحوا فرع في جده,اصيل,M
7402,1099028431931691008,2019-02-22 13:29:31,http://twitter.com/FaisalDawood16/status/1099028431931691008,عزيزي، البيتزا شي 🇮🇹 والمفطح شي 🇸🇦 مو لازم نخلط الامور!! #لغوصه #مايسترو #بيتزا #ثقف_نفسك,@FaisalDawood16,Faisal Dawood,NaN,NaN,NaN,Negative,Unclassified,Twitter,M,92,20,145,NaN,عزيزي،\n البيتزا شي 🇮🇹\nوالمفطح شي 🇸🇦\nمو لازم نخلط الامور!!\n #لغوصه #مايسترو #بيتزا #ثقف_نفسك,Tweet,NaN,NaN,عزيزي البيتزا شي 🇮🇹 والمفطح شي 🇸🇦 مو لازم نخلط الامور لغوصه مايسترو بيتزا ثقفنفسك,faisal,M
7403,1093822031492275968,2019-02-08 04:41:09,http://twitter.com/B_vlinder/status/1093822031492276224,@zeze00042 @CareemKSA طريقة الطلب اختاروا فئة بيتزا نقطة الانطلاق مايسترو بيتزا طبعا تختاروا أقرب فرع لكم أو بكيفكم الفرع اللي تبون بعدين نقطة الوصول اللي هي مكان تواجدك بيطلع لك القيمة صفر والله يرزقني ويرزقكم من واسع فضله,@B_vlinder,B.vlinder,Saudi Arabia,Ar Riyad,Riyadh,Neutral,Unclassified,Twitter,NaN,3312,270,578,NaN,@zeze00042 @CareemKSA طريقة الطلب \nاختاروا فئة بيتزا \nنقطة الانطلاق مايسترو بيتزا طبعا تختاروا أقرب فرع لكم أو بكيفكم الفرع اللي تبون\nبعدين نقطة الوصول اللي هي مكان تواجدك \nبيطلع لك القيمة صفر \nوالله يرزقني ويرزقكم من واسع فضله,Reply,NaN,NaN,zeze042 caremksa طريقه الطلب اختاروا فئه بيتزا نقطه الانطلاق مايسترو بيتزا طبعا تختاروا اقرب فرع لكم او بكيفكم الفرع الي تبون بعدين نقطه الوصول الي هي مكان تواجدك بيطلع لك القيمه صفر واله يرزقني ويرزقكم من واسع فضله,bvlinder,Known
7404,1103387821237304960,2019-03-06 14:12:11,http://twitter.com/noooormd84404/status/1103387821237305344,@Hanan3omr @LaughWithMedia @MaestroPizzaKSA هذه سخنوها مكرويف,@noooormd84404,غربة ♋️ إحساس,Saudi Arabia,NaN,NaN,Neutral,Unclassified,Twitter,NaN,10346,456,360,NaN,@Hanan3omr @LaughWithMedia @MaestroPizzaKSA هذه سخنوها مكرويف,Reply,NaN,NaN,hanan3omr laughwithmedia maestropizaksa هذه سخنوها مكرويف,غربه,Known
7405,1104739997058507008,2019-03-10 08:45:15,http://twitter.com/hantamy55/status/1104739997058506754,@crazy_zoon @LaughWithMedia @MaestroPizzaKSA @Saad_Maestro حاجة تكسف..!! https://t.co/7kbI5K1XlW,@hantamy55,ابو عبدالرحمن,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,21919,235,490,NaN,@crazy_zoon @LaughWithMedia @MaestroPizzaKSA @Saad_Maestro حاجة تكسف..!! https://t.co/7kbI5K1XlW,Reply,NaN,NaN,crazyzon laughwithmedia maestropizaksa sadmaestro حاجه تكسف,ابو,Known
7406,1103404175344520960,2019-03-06 15:17:10,http://twitter.com/sh_ib77/status/1103404175344521216,RT @HungerStation لا تخاطر مع الجوع ✋🏼 اطلب من المضمون واللي تحبه 🍔🍟 فايف جايز يوصلك من #هنقرستيشن بضغطة زر وبدون رسوم التوصيل 🤳🏼💛 https://t.co/yEMDoZO2rS https://t.co/v0DeadW35G,@sh_ib77,شيٰفہٰشيٰنكہٰو,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,35468,1317,1866,NaN,RT @HungerStation لا تخاطر مع الجوع ✋🏼 اطلب من المضمون واللي تحبه 🍔🍟\n فايف جايز يوصلك من #هنقرستيشن بضغطة زر وبدون رسوم التوصيل 🤳🏼💛\nhttps://t.co/yEMDoZO2rS https://t.co/v0DeadW35G,Retweet,NaN,NaN,rt hungerstation لا تخاطر مع الجوع ✋🏼 اطلب من المضمون والي تحبه 🍔🍟 فايف جايز يوصلك من ه

In [29]:
len(tweets)

137776

In [30]:
tweets.loc[7400:7500]

,GUID,Date (CST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,Emotion,Source,Gender,Posts,Followers,Following,Influence Score,Post Title,Post Type,Image URL,Brand,clean text,clean Name,gender1
7400,1097550482552381056,2019-02-18 11:36:41,http://twitter.com/fi11ii/status/1097550482552381440,@Taghroooda فيه الجلسات حول الرياض بارك حلوه .. ستيك هاوس .. مادو الي فالتحليه فتح .. سنتريا .. اضواء المساء,@fi11ii,باتريك #الشاهي☕️,Saudi Arabia,Ar Riyad,Riyadh,Positive,Unclassified,Twitter,NaN,2582,73,401,NaN,@Taghroooda فيه الجلسات حول الرياض بارك حلوه .. ستيك هاوس .. مادو الي فالتحليه فتح .. سنتريا .. اضواء المساء,Reply,NaN,NaN,taghroda فيه الجلسات حول الرياض بارك حلوه ستيك هاوس مادو الي فالتحليه فتح سنتريا اضواء المساء,باتريك,Known
7401,1094545436239646976,2019-02-10 04:35:42,http://twitter.com/A9eel82/status/1094545436239646720,@HamburginiSA ياخي افتحوا فرع في جدة,@A9eel82,اصيل,Saudi Arabia,Makkah,Jeddah,Neutral,Unclassified,Twitter,NaN,2114,93,510,NaN,@HamburginiSA ياخي افتحوا فرع في جدة,Reply,NaN,NaN,hamburginisa ياخي افتحوا فرع في جده,اصيل,M
7402,1099028431931691008,2019-02-22 13:29:31,http://twitter.com/FaisalDawood16/status/1099028431931691008,عزيزي، البيتزا شي 🇮🇹 والمفطح شي 🇸🇦 مو لازم نخلط الامور!! #لغوصه #مايسترو #بيتزا #ثقف_نفسك,@FaisalDawood16,Faisal Dawood,NaN,NaN,NaN,Negative,Unclassified,Twitter,M,92,20,145,NaN,عزيزي،\n البيتزا شي 🇮🇹\nوالمفطح شي 🇸🇦\nمو لازم نخلط الامور!!\n #لغوصه #مايسترو #بيتزا #ثقف_نفسك,Tweet,NaN,NaN,عزيزي البيتزا شي 🇮🇹 والمفطح شي 🇸🇦 مو لازم نخلط الامور لغوصه مايسترو بيتزا ثقفنفسك,faisal,M
7403,1093822031492275968,2019-02-08 04:41:09,http://twitter.com/B_vlinder/status/1093822031492276224,@zeze00042 @CareemKSA طريقة الطلب اختاروا فئة بيتزا نقطة الانطلاق مايسترو بيتزا طبعا تختاروا أقرب فرع لكم أو بكيفكم الفرع اللي تبون بعدين نقطة الوصول اللي هي مكان تواجدك بيطلع لك القيمة صفر والله يرزقني ويرزقكم من واسع فضله,@B_vlinder,B.vlinder,Saudi Arabia,Ar Riyad,Riyadh,Neutral,Unclassified,Twitter,NaN,3312,270,578,NaN,@zeze00042 @CareemKSA طريقة الطلب \nاختاروا فئة بيتزا \nنقطة الانطلاق مايسترو بيتزا طبعا تختاروا أقرب فرع لكم أو بكيفكم الفرع اللي تبون\nبعدين نقطة الوصول اللي هي مكان تواجدك \nبيطلع لك القيمة صفر \nوالله يرزقني ويرزقكم من واسع فضله,Reply,NaN,NaN,zeze042 caremksa طريقه الطلب اختاروا فئه بيتزا نقطه الانطلاق مايسترو بيتزا طبعا تختاروا اقرب فرع لكم او بكيفكم الفرع الي تبون بعدين نقطه الوصول الي هي مكان تواجدك بيطلع لك القيمه صفر واله يرزقني ويرزقكم من واسع فضله,bvlinder,Known
7404,1103387821237304960,2019-03-06 14:12:11,http://twitter.com/noooormd84404/status/1103387821237305344,@Hanan3omr @LaughWithMedia @MaestroPizzaKSA هذه سخنوها مكرويف,@noooormd84404,غربة ♋️ إحساس,Saudi Arabia,NaN,NaN,Neutral,Unclassified,Twitter,NaN,10346,456,360,NaN,@Hanan3omr @LaughWithMedia @MaestroPizzaKSA هذه سخنوها مكرويف,Reply,NaN,NaN,hanan3omr laughwithmedia maestropizaksa هذه سخنوها مكرويف,غربه,Known
7405,1104739997058507008,2019-03-10 08:45:15,http://twitter.com/hantamy55/status/1104739997058506754,@crazy_zoon @LaughWithMedia @MaestroPizzaKSA @Saad_Maestro حاجة تكسف..!! https://t.co/7kbI5K1XlW,@hantamy55,ابو عبدالرحمن,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,21919,235,490,NaN,@crazy_zoon @LaughWithMedia @MaestroPizzaKSA @Saad_Maestro حاجة تكسف..!! https://t.co/7kbI5K1XlW,Reply,NaN,NaN,crazyzon laughwithmedia maestropizaksa sadmaestro حاجه تكسف,ابو,Known
7406,1103404175344520960,2019-03-06 15:17:10,http://twitter.com/sh_ib77/status/1103404175344521216,RT @HungerStation لا تخاطر مع الجوع ✋🏼 اطلب من المضمون واللي تحبه 🍔🍟 فايف جايز يوصلك من #هنقرستيشن بضغطة زر وبدون رسوم التوصيل 🤳🏼💛 https://t.co/yEMDoZO2rS https://t.co/v0DeadW35G,@sh_ib77,شيٰفہٰشيٰنكہٰو,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,35468,1317,1866,NaN,RT @HungerStation لا تخاطر مع الجوع ✋🏼 اطلب من المضمون واللي تحبه 🍔🍟\n فايف جايز يوصلك من #هنقرستيشن بضغطة زر وبدون رسوم التوصيل 🤳🏼💛\nhttps://t.co/yEMDoZO2rS https://t.co/v0DeadW35G,Retweet,NaN,NaN,rt hungerstation لا تخاطر مع الجوع ✋🏼 اطلب من المضمون والي تحبه 🍔🍟 فايف جايز يوصلك من ه

In [31]:
listOfrest=[['outback steakhouse','أوت باك ستيك هاوس'],
 ['bcskudar steak house','أسكودار ستيك هاوس'],
 ['sultans steakhouse','سلطان ستيك هاوس'],
 ['spread','سبرد'],
 ['texas roadhouse','تكساس رود هاوس', 'تكساس رودهاوس'],
 ['back yard gril','باك يارد جريل',
  'backyardgrilsa'],
 ['burgerfuel','برجر فيول',
  'burgerfuelksa',
  'برجرفيول',
  'برقر فيول',
  'برغر فيول',
  'برغرفيول'],
 ['porterhouse','بورتر هاوس',
  'porterhousesa'],
 ['steak house','ستيك هاوس',
  'steakhouseksa'],
 ['five guys','فايف جايز', 
  'fiveguysbahrain', 
  'فايف قايز'],
 ['wagyu  the elite','واقيو برجر',
  'واجيو برجر',
  "برجر واجيو",
  "واقيو",
  'wagyuburger',
  'wagyu_burger',
  'واقيو برقر',
  'واقيو برغر',
 "wagyuburger"],
 ['salt','سولت'],
 ['burger boutique','برجر بوتيك', 
  'burgerboutique',
  "بوتيك","برغر بوتيك"
  ,"برقر بوتيك"],          
 ['the real burger','ريل برجر',
  'realburgersa',
  "ريل برقر"],                              
 ['hamburgini','هامبرغيني',
  'هامبرقيني',
  'همبرغيني',
  'هابرغيني',
  'hamburginisa',
  'هامبرجيني'],
 ['sectionb','سكشن-بي',
  'سكشن بي',
  'sectionbsa',
  "سكشين بي",
  "sectionb",
  "بي سكشن"],
 ['32 grimp','32grimp',
  'قريب32','32قريب'
  ,'32 قرمب'],      
 ['shrimp shack','شرمبشاك', 
  'shrimpshacksa', 
  'شرمب شاك'],
 ['اسماك دله',
  'اسماك دلة'],
 ['shrimplus','شريمبلس ',
  "شريمبلس",
  'shrimplus'],
 ['سمكيات'],
 ['pf changs','بي.اف. تشانغز',
  'بي اف تشانغز'],
 ['tokyo','طوكيو'],
 ['nozomi','نوزومي',
  'نزومي',
  'مطعم نوزومي'],
 ['yokari','يوكاري',
  'yokaririyadh',
  'مطعم يوكاري'],
 ['benihana','بنيهانا'],
 ['sushi yoshi','سوشي يوشي'],
 ['yummy wok','يمي ووك',
 "يمي وك"],
 ['wok to walk','مطعم ووك تو ووك',
  'مطعم ووك تو والك',
  'WoktoWalkMe'],
 ['noodles factory','نودلز فاكتوري', 
  'noodlesfactory1'],
 ['wokkong','ووك كونج'],
 ['maestro piza','مايسترو بيتزا',
  "مايسترو",
  'maestropizaksa',
  'ماسترو بيتزا',
  'maestropizaksa',
  'مايستر بيتزا',
  'مايستروبيتزا',
  'مطعم مايسترو بيتزا'],
 ['blaze piza','lets piza','بليز بيتزا',
 "بليز بيتزا"],
 ['lets piza','ليتس بيتزا', 
  'letspizasa',
  'لتس بيتزا'],
 ['pizarati','بيتزاراتي', 
  'pizarati',
  "Pizarati"],
 ['boga','مطعم بوقا','بوقا'],
 ['subway','صب واي',
  'مطعم صب واي',
  'subwaysaudi',
  'مطعم صبواي',
  "صب و اي"
  "صبواي"],
 ['mama noura','ماما نوره', 
  'مطعم مامانوره',
  'مامانوره', 
  'مطعم ماما نوره',
 "ماما نورة"],
 ['which wich','ويتش وتش',
  'whichWichoman']]

In [32]:
count=0
def typerestruant(tweet):
    rest=[]

    for lis in listOfrest:
        for l in lis:
            if l in tweet:
                rest.append(lis[0])

    if len(rest) ==0:
        rest.append("None")

    return rest

In [33]:
if("@SectionB_sa"=='@SectionB_sa'.lower):
    print(True)

In [34]:
print(typerestruant("الشهر اللي فات اكلنا مرتين كنتاكي ومرتين مايسترو بيتزا المفروض هنقر ستيشن يعملوله تخفيض"))

['maestro piza', 'maestro piza']


In [35]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137776 entries, 0 to 137775
Data columns (total 24 columns):
GUID               137776 non-null int64
Date (CST)         137776 non-null datetime64[ns]
URL                62886 non-null object
Contents           137760 non-null object
Author             137776 non-null object
Name               137758 non-null object
Country            61139 non-null object
State/Region       40235 non-null object
City/Urban Area    37035 non-null object
Category           137776 non-null object
Emotion            137776 non-null object
Source             137776 non-null object
Gender             24928 non-null object
Posts              137776 non-null int64
Followers          137776 non-null int64
Following          137776 non-null int64
Influence Score    14664 non-null float64
Post Title         137760 non-null object
Post Type          137776 non-null object
Image URL          0 non-null float64
Brand              723 non-null object
clean text     

In [36]:
tweets.drop(tweets[tweets['Contents']==0].index,inplace=True)
tweets.reset_index(drop=True,inplace=True)

In [37]:
type(tweets.Contents)

pandas.core.series.Series

In [38]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137775 entries, 0 to 137774
Data columns (total 24 columns):
GUID               137775 non-null int64
Date (CST)         137775 non-null datetime64[ns]
URL                62885 non-null object
Contents           137759 non-null object
Author             137775 non-null object
Name               137757 non-null object
Country            61139 non-null object
State/Region       40235 non-null object
City/Urban Area    37035 non-null object
Category           137775 non-null object
Emotion            137775 non-null object
Source             137775 non-null object
Gender             24928 non-null object
Posts              137775 non-null int64
Followers          137775 non-null int64
Following          137775 non-null int64
Influence Score    14664 non-null float64
Post Title         137759 non-null object
Post Type          137775 non-null object
Image URL          0 non-null float64
Brand              723 non-null object
clean text     

In [39]:
tweets['typerestruant']=tweets['clean text'].apply(lambda x : typerestruant(str(x)))

In [40]:
tweets['typerestruant'].value_counts()

[maestro piza, maestro piza]                                                                                                                                                                                                           59932
[subway, subway]                                                                                                                                                                                                                       9623 
[mama noura]                                                                                                                                                                                                                           9530 
[subway]                                                                                                                                                                                                                               6570 
[steak house]                                       

In [41]:
tweets.head(90)

,GUID,Date (CST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,Emotion,Source,Gender,Posts,Followers,Following,Influence Score,Post Title,Post Type,Image URL,Brand,clean text,clean Name,gender1,typerestruant
0,1081815582889176960,2019-01-06 01:31:48,http://twitter.com/aboaljod59/status/1081815582889177088,@aboaljod59 @LuayPrince هذه المعلومات الي شفتها حق صب واي بالجامعة https://t.co/Gj9OwBYV1q,@aboaljod59,aboaljod59,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,547,35,376,0.010342,@aboaljod59 @LuayPrince هذه المعلومات الي شفتها حق صب واي بالجامعة https://t.co/Gj9OwBYV1q,Reply,NaN,NaN,aboaljod59 luayprince هذه المعلومات الي شفتها حق صب واي بالجامعه,aboaljod59,Known,[subway]
1,1085259913788576000,2019-01-15 13:38:21,http://twitter.com/noudikh/status/1085259913788575744,@tariq_man_ @sectionb_sa اففف ياحلوو البرجر حقهممم 💔💔💔,@noudikh,العنود المساعد💙🐣,Saudi Arabia,Ar Riyad,Riyadh,Neutral,Unclassified,Twitter,NaN,32286,789,361,NaN,@tariq_man_ @sectionb_sa اففف ياحلوو البرجر حقهممم 💔💔💔,Reply,NaN,NaN,tariqman sectionbsa اف ياحلو البرجر حقهم 💔,العنود,F,"[sectionb, sectionb, sectionb]"
2,1092097568463565056,2019-02-03 10:28:45,http://twitter.com/__muha97/status/1092097568463564800,صب واي #ماك_او_كودو,@__muha97,نسّاي,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,1434,115,470,NaN,صب واي \n #ماك_او_كودو,Tweet,NaN,NaN,صب واي ماكاوكودو,نس اي,Known,[subway]
3,1090908449523814016,2019-01-31 03:43:37,http://twitter.com/abuhussien20201/status/1090908449523814400,RT @SubwaySaudi ودك بشيء خفيف ولذيذ 😋 #صب_واى_السعودية بانتظارك 😎👌 https://t.co/gI7DjXAyLy,@abuhussien20201,abuhussien2020,NaN,NaN,NaN,Positive,Unclassified,Twitter,NaN,67310,634,2027,NaN,RT @SubwaySaudi ودك بشيء خفيف ولذيذ 😋 #صب_واى_السعودية بانتظارك \n😎👌 https://t.co/gI7DjXAyLy,Retweet,NaN,NaN,rt subwaysaudi ودك بشيء خفيف ولذيذ 😋 صبوايالسعوديه بانتظارك 😎👌,abuhussien2020,Known,"[subway, subway]"
4,1082695506277613952,2019-01-08 11:48:18,http://twitter.com/monakhaled88/status/1082695506277613571,@MaestroPizzaKSA #مسترها تم https://t.co/MO7CpgudYC,@monakhaled88,منى بنت خالد,Saudi Arabia,Ar Riyad,Riyadh,Neutral,Unclassified,Twitter,NaN,9736,83,510,NaN,@MaestroPizzaKSA #مسترها تم https://t.co/MO7CpgudYC,Reply,NaN,NaN,maestropizaksa مسترها تم,مني,F,"[maestro piza, maestro piza]"
5,1081856809881752064,2019-01-06 04:15:38,http://twitter.com/gy0__gy0/status/1081856809881751552,@LuayPrince طيب كانوا يقولون صب واي مافيه ولا سعرة حرارية واللي يطلع أي شي له الوجبة مجانا !!,@gy0__gy0,#تميمي,NaN,NaN,NaN,Positive,Unclassified,Twitter,NaN,10128,607,445,NaN,@LuayPrince طيب كانوا يقولون صب واي مافيه ولا سعرة حرارية واللي يطلع أي شي له الوجبة مجانا !!,Reply,NaN,NaN,luayprince طيب كانوا يقولون صب واي مافيه ولا سعره حراريه والي يطلع اي شي له الوجبه مجانا,تميمي,Known,[subway]
6,1083700402766283008,2019-01-11 06:21:24,http://twitter.com/nowetesh/status/1083700402766282752,#شركة_نويتش لبيع لوازم الرحلات البرية والبحرية العنوان/ #الكويت منطقة الشويخ الصناعية الثالثة شارع المخفر القديم مقابل مطعم صب واي ومحطة كارسيرفس لغسيل السيارات محل رقم ٦ يوجد… https://t.co/ydXChw2ZqD,@nowetesh,نويتش,NaN,NaN,NaN,Neutral,Unclassified,Twitter,NaN,1365,342,611,NaN,#شركة_نويتش \nلبيع لوازم الرحلات البرية والبحرية\n\nالعنوان/\n\n#الكويت منطقة الشويخ الصناعية الثالثة \nشارع المخفر القديم مقابل مطعم صب واي ومحطة كارسيرفس لغسيل السيارات\n\nمحل رقم ٦\n\nيوجد… https://t.co/ydXChw2ZqD,Tweet,NaN,NaN,شركهنويتش لبيع لوازم الرحلات البريه والبحريه العنوان الكويت منطقه الشويخ الصناعيه الثالثه شارع المخفر القديم مقابل مطعم صب واي ومحطه كارسيرفس لغسيل السيارات محل رقم ٦ يوجد,نويتش,Known,"[subway, subway]"
7,1081977806974796032,2019-01-06 12:16:26,http://twitter.com/Amal_Alfaisl/status/1081977806974795776,@atharoly ساندوتش صدور دجاج من صب واي,@Amal_Alfaisl,امل الفيصل,Saudi Arabia,Ash Sharqiyah,Ad Damman,Neutral,Unclassified,Twitter,NaN,29533,6074,252,NaN,@atharoly ساندوتش صدور دجاج من صب واي,Reply,NaN,NaN,atharoly ساندوتش صدور دجاج من صب واي,امل,F,[subway]
8,1089949639653147008,2019-01-28 12:13:39,http://twitter.com/M_A_

In [42]:
len(tweets['typerestruant'][0])

1

In [43]:
len(tweets["typerestruant"][3234])

1

In [48]:
tweets[tweets['typerestruant']=="wok to walk"]

,GUID,Date (CST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,Emotion,Source,Gender,Posts,Followers,Following,Influence Score,Post Title,Post Type,Image URL,Brand,clean text,clean Name,gender1,typerestruant


In [44]:
def removedup(li):#remove of duplict resturant 
    rest=""
    li = set(li)
    li= list(li)

    return li

In [45]:
tweets['typerestruant']=tweets['typerestruant'].apply(lambda x : removedup(x))

In [46]:
tweets['typerestruant'].value_counts()

[maestro piza]                                                                           63734
[subway]                                                                                 16696
[mama noura]                                                                             11487
[steak house]                                                                            6785 
[hamburgini]                                                                             5772 
[pizarati]                                                                               4386 
[five guys]                                                                              3609 
[sectionb]                                                                               3521 
[burger boutique]                                                                        3217 
[nozomi]                                                                                 2093 
[32 grimp]                                        

In [ ]:
test=tweets

In [ ]:
len(test)

In [ ]:
typee=test["typerestruant"]

In [ ]:
typee=typee.tolist()

In [ ]:
len(typee)

In [ ]:
#if any row has more than one resturant put each resturant in different row in 
dele=[]
index=[]
typ=[]
for li in range(len(typee)):
    if(len(typee[li])>1):
        dele.append(li)
        for l in range(len(typee[li])):
            typ.append(typee[li][l])
            index.append(li)
    else:
        typ.append(typee[li][0])
        index.append(li)
    

In [ ]:
index[84]

In [ ]:
df=pd.DataFrame()


In [ ]:
len(typ)

In [ ]:
len(df)
len(index)

In [ ]:
for i in range(len(typ)):#to copy the all rows have need it to copy 
    print(i)
    df=df.append(test[test.index==index[i]].copy())

In [ ]:
len(typ)

In [ ]:
len(df)

In [ ]:
df["typerestruant"]=typ #now there is one resurant in row 

In [ ]:
dele

In [ ]:
df.loc[7000:8000]

In [ ]:
df.drop(df[df["typerestruant"]=="None"].index,inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)